In [ ]:
import requests as req
import json
import math
import os

import matplotlib.pyplot as plt
import seaborn as sns

# Get metadata from Dataverse for each DOI

In [ ]:
from string import Template
query_base=Template('https://dataverse.harvard.edu/api/search?q="${doi}"&show_facets=true')

In [ ]:
import pandas
unique_dois = pandas.read_csv("../get-dois/dataset_dois.txt", names=['DOI'])

In [ ]:
unique_dois.head()

In [ ]:
unique_dois = unique_dois['DOI'].values.tolist()

In [ ]:
len(unique_dois)

In [ ]:
query = query_base.substitute(doi = unique_dois[0])
res = req.get(query)
res_dict = json.loads(res.text)

In [ ]:
time = str(res_dict['data']['items'][0]['published_at'][:4])
time

In [ ]:
all_metadata=""
all_metadata = ','.join(["DOI","publicationDate","publisher","subject"]) + os.linesep
for i,doi in enumerate(unique_dois):
        query = query_base.substitute(doi = doi)

        res = req.get(query)
        res_dict = json.loads(res.text)
        try:
            time = res_dict['data']['items'][0]['published_at'][:4]
        except IndexError:
            print res_dict['data']['items']
            
        try:
            subs = res_dict['data']['facets'][0]['subject_ss']['labels']
            subject = ";".join(str(s.keys()[0]) for s in subs)
        except KeyError:
            subs = "NA"
        
        try:
            publisher = res_dict['data']['items'][0]['identifier_of_dataverse']
        except:
            print doi
            print res_dict['data']
            
        metadata_line = ','.join([doi.strip(), time, publisher.replace(",",";"), subject.replace(",",";")])
        #print metadata_line
        all_metadata += metadata_line + os.linesep

In [ ]:
import pandas as pd 
from StringIO import StringIO

df = pd.read_csv(StringIO(all_metadata))
#df = pd.read_csv(StringIO(all_metadata), delimiter=',')
df.head()

In [ ]:
len(df)

In [ ]:
with open('all_metadata_sept.txt','w') as f:
    f.write(all_metadata.encode('utf8'))

# Get publisher metadata from Dataverse

In [ ]:
import pandas as pd
metadata = pd.read_csv("all_metadata_sept.txt")
metadata.head()

In [ ]:
unique = metadata['publisher'].unique()
unique[:10]

In [ ]:
from string import Template
import requests as req
import json
import os

qbase=Template('https://dataverse.harvard.edu/api/dataverses/${iddv}')

all_journals = ""
for i,iddv in enumerate(unique):
    query = qbase.substitute(iddv = iddv)
    res = req.get(query)
    res_dict = json.loads(res.text)
    
    try:
        name = res_dict['data']['name']
        dvtype = res_dict['data']['dataverseType']
        
        metadata_line = '\t'.join([iddv.strip(), dvtype, name])
        #print metadata_line
        all_journals += metadata_line + os.linesep
    except:
        print res_dict

In [ ]:
from StringIO import StringIO
dj = pd.read_csv(StringIO(all_journals), sep='\t', names=['publisher', 'type', 'name'])

dj.head()

In [ ]:
len(metadata)

In [ ]:
len(dj)

In [ ]:
merged = pd.merge(metadata, dj, on='publisher')

In [ ]:
len(merged)

In [ ]:
merged.head()

In [ ]:
merged.to_csv('metadata_merged.csv')